In [1]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
import os
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-zmadureira1/code/Users/s-zmadureira/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')
img_size=150

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=5,
        zoom_range = 0.1 ,
        horizontal_flip=False,
        channel_shift_range=.001,
        preprocessing_function= lambda x: tf.image.rgb_to_hsv(x)
    )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
                      batch_size=4,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(img_size, img_size),
        batch_size=4,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        batch_size=4,
        class_mode='binary')


model = models.Sequential()
model.add(layers.Conv2D(128, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(128, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())

model.add(layers.Dense(512, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True,
        name="Adam"
        )


model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC'),metrics.FalsePositives(),
                       metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])

from keras.callbacks import ModelCheckpoint

from keras.callbacks import ReduceLROnPlateau


checkpoint = ModelCheckpoint('checkpoint_best_model_HSV_30epoch.h5', monitor='val_recall', verbose=1,
                             save_best_only=True, mode='max')

# Add the callback to your list of callbacks
callbacks_list = [checkpoint]

history = model.fit(
        train_generator,
        epochs=30,
        validation_data=validation_generator,
          callbacks=[callbacks_list],
                       verbose=1)
#model.save('supermodel_final.h5')


2023-04-07 22:47:32.582126: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-07 22:47:47.817527: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-07 22:47:47.817633: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-07 22:47:47.817644: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Num GPUs Available:  1
Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/30
3042/3042 [==============================] - ETA: 0s - loss: 0.6443 - recall: 0.6158 - precision: 0.6434 - accuracy: 0.6537 - AUC: 0.6951 - PR-AUC: 0.6464 - false_positives: 1982.0000 - false_negatives: 2231.0000 - true_negatives: 4377.0000 - true_positives: 3576.0000
Epoch 1: val_recall improved from -inf to 0.01592, saving model to checkpoint_best_model_HSV_30epoch.h5
3042/3042 [==============================] - 2198s 708ms/step - loss: 0.6443 - recall: 0.6158 - precision: 0.6434 - accuracy: 0.6537 - AUC: 0.6951 - PR-AUC: 0.6464 - false_positives: 1982.0000

In [2]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-zmadureira1/code/Users/s-zmadureira/models/model_HSV_30epoch.h5')

In [3]:
loss, recall, precision, accuracy, auc ,pr_auc, fp, fn, tn,tp = model.evaluate(test_generator)
print('Test loss:', loss)
print('Test recall:', recall)
print('Test recall:', precision)
print('Test accuracy:', accuracy)
print('Test auc:', auc)
print('Test pr_auc:', pr_auc)
print('Test fp:', fp)
print('Test fn:', fn)
print('Test tn:', tn)
print('Test tp:', tp)
#model_HSV_30epoch ratings

381/381 [==============================] - 204s 535ms/step - loss: 10877238.0000 - recall: 0.0000e+00 - precision: 0.0000e+00 - accuracy: 0.5628 - AUC: 0.5000 - PR-AUC: 0.4372 - false_positives: 0.0000e+00 - false_negatives: 665.0000 - true_negatives: 856.0000 - true_positives: 0.0000e+00
Test loss: 10877238.0
Test recall: 0.0
Test recall: 0.0
Test accuracy: 0.562787652015686
Test auc: 0.5
Test pr_auc: 0.43721234798431396
Test fp: 0.0
Test fn: 665.0
Test tn: 856.0
Test tp: 0.0


### THIS IS THE HEAVY LAYERED MODEL!!!!!!!!!!!!! USED IN THESIS

In [1]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
import os
import cv2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-zmadureira1/code/Users/s-zmadureira/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')


img_size=512
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    channel_shift_range=0.001
)


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
                      batch_size=8,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(img_size, img_size),
        batch_size=8,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        batch_size=8,
        class_mode='binary')

regularizer = l2(0.001)


model = models.Sequential()
model.add(layers.Conv2D(32, (5,5), activation='relu',kernel_regularizer=regularizer,  input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(32, (5, 5), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(128, (3, 3), activation='relu',kernel_regularizer=regularizer))

model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(256, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(256, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.Conv2D(512, (3, 3), activation='relu',kernel_regularizer=regularizer))
model.add(layers.BatchNormalization())
model.add(layers.MaxPooling2D((2, 2)))



model.add(layers.Flatten())
model.add(layers.Dense(512, activation="relu",kernel_regularizer=regularizer))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid',kernel_regularizer=regularizer))
from keras.callbacks import ModelCheckpoint

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
        learning_rate=2e-5,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True,
        name="Adam")

checkpoint = ModelCheckpoint('checkpoint_best_weirdmodel_25epoch_512pix.h5', monitor='val_recall', verbose=1,
                             save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model.compile(optimizer=opt,loss='binary_crossentropy',metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),metrics.AUC(curve='PR', name = 'PR-AUC')
                       ,metrics.FalsePositives(), metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])



history = model.fit(
        train_generator,
        epochs=20,
        validation_data=validation_generator,callbacks=[callbacks_list],
                       verbose=1)


2023-04-09 12:28:52.823784: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-09 12:29:14.690029: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-09 12:29:14.690141: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-09 12:29:14.690154: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

Num GPUs Available:  1
Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/20
1521/1521 [==============================] - ETA: 0s - loss: 3.7672 - recall: 0.6819 - precision: 0.6463 - accuracy: 0.6701 - AUC: 0.7259 - PR-AUC: 0.6554 - false_positives: 2167.0000 - false_negatives: 1847.0000 - true_negatives: 4192.0000 - true_positives: 3960.0000
Epoch 1: val_recall improved from -inf to 0.63926, saving model to checkpoint_best_weirdmodel_25epoch_512pix.h5
1521/1521 [==============================] - 2434s 2s/step - loss: 3.7672 - recall: 0.6819 - precision: 0.6463 - accuracy: 0.6701 - AUC: 0.7259 - PR-AUC: 0.6554 - false_positives: 2167

In [2]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-zmadureira1/code/Users/s-zmadureira/models/checkpoint_best_weirdmodel_25epoch_512pix.h5')

In [3]:
loss, recall, precision, accuracy, auc ,pr_auc, fp, fn, tn,tp = model.evaluate(test_generator)
print('Test loss:', loss)
print('Test recall:', recall)
print('Test recall:', precision)
print('Test accuracy:', accuracy)
print('Test auc:', auc)
print('Test pr_auc:', pr_auc)
print('Test fp:', fp)
print('Test fn:', fn)
print('Test tn:', tn)
print('Test tp:', tp)
#supergigamodel_clahe_25epoch_512pix ratings


191/191 [==============================] - 206s 1s/step - loss: 2.8181 - recall: 0.7699 - precision: 0.7817 - accuracy: 0.8054 - AUC: 0.8792 - PR-AUC: 0.8322 - false_positives: 143.0000 - false_negatives: 153.0000 - true_negatives: 713.0000 - true_positives: 512.0000
Test loss: 2.8181052207946777
Test recall: 0.7699248194694519
Test recall: 0.7816793918609619
Test accuracy: 0.8053911924362183
Test auc: 0.8791616559028625
Test pr_auc: 0.8322091698646545
Test fp: 143.0
Test fn: 153.0
Test tn: 713.0
Test tp: 512.0


In [4]:
ws = Workspace.from_config()
compute_target = ws.compute_targets['s-zmadureira1']

compute_target
compute_target.stop(wait_for_completion=True)

NameError: name 'Workspace' is not defined

In [4]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
import os
from keras.callbacks import ModelCheckpoint

import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-zmadureira1/code/Users/s-zmadureira/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')

img_size=150

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=5,
        zoom_range = 0.1 ,
        horizontal_flip=False,
        channel_shift_range=.001
        #preprocessing_function= lambda x: tf.image.rgb_to_hsv(x)
    )

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
                      batch_size=8,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validate_dir,
        target_size=(img_size, img_size),
        batch_size=8,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        batch_size=8,
        class_mode='binary')


model = models.Sequential()
model.add(layers.Conv2D(128, (5, 5), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(layers.Conv2D(128, (5, 5), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.Conv2D(512, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.Conv2D(1024, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(2048, (3, 3), activation='relu'))
#model.add(layers.Conv2D(2048, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))


model.add(layers.Flatten())
model.add(layers.Dense(2048, activation="relu"))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1024, activation="relu"))
model.add(layers.BatchNormalization())
model.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
        learning_rate=2e-5,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-07,
        amsgrad=True,
        name="Adam")

model.compile(optimizer=opt,
                  loss='binary_crossentropy',
                  metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),
                       metrics.AUC(curve='PR', name = 'PR-AUC'),metrics.FalsePositives(),
                       metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])


checkpoint = ModelCheckpoint('checkpoint_bigmodel_25epoch_150pix.h5', monitor='val_recall_2', verbose=1,
                             save_best_only=True, mode='max')

callbacks_list = [checkpoint]

history = model.fit(
        train_generator,
        epochs=25,
        validation_data=validation_generator,
                       verbose=1,callbacks=[checkpoint])
#model.save('supermodel_final.h5')


Num GPUs Available:  1
Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/25
1521/1521 [==============================] - ETA: 0s - loss: 0.6103 - recall_2: 0.6465 - precision_2: 0.6618 - accuracy: 0.6736 - AUC: 0.7320 - PR-AUC: 0.6888 - false_positives_2: 1918.0000 - false_negatives_2: 2053.0000 - true_negatives_2: 4441.0000 - true_positives_2: 3754.0000
Epoch 1: val_recall_2 improved from -inf to 0.45756, saving model to checkpoint_bigmodel_25epoch_150pix.h5
1521/1521 [==============================] - 1772s 1s/step - loss: 0.6103 - recall_2: 0.6465 - precision_2: 0.6618 - accuracy: 0.6736 - AUC: 0.7320 - PR-AUC: 0.6888 

In [5]:
model.save('/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-zmadureira1/code/Users/s-zmadureira/models/bigmodel_25epoch_299pix.h5')

In [6]:
loss, recall, precision, accuracy, auc ,pr_auc, fp, fn, tn,tp = model.evaluate(test_generator)
print('Test loss:', loss)
print('Test recall:', recall)
print('Test recall:', precision)
print('Test accuracy:', accuracy)
print('Test auc:', auc)
print('Test pr_auc:', pr_auc)
print('Test fp:', fp)
print('Test fn:', fn)
print('Test tn:', tn)
print('Test tp:', tp)
#bigmodel_25epoch_299pix ratings


191/191 [==============================] - 205s 1s/step - loss: 0.7023 - recall_2: 0.6165 - precision_2: 0.8350 - accuracy: 0.7791 - AUC: 0.8755 - PR-AUC: 0.8352 - false_positives_2: 81.0000 - false_negatives_2: 255.0000 - true_negatives_2: 775.0000 - true_positives_2: 410.0000
Test loss: 0.7022920846939087
Test recall: 0.61654132604599
Test recall: 0.8350305557250977
Test accuracy: 0.7790927290916443
Test auc: 0.8754575848579407
Test pr_auc: 0.8351903557777405
Test fp: 81.0
Test fn: 255.0
Test tn: 775.0
Test tp: 410.0


In [9]:
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import os
base_dir = "/mnt/batch/tasks/shared/LS_root/mounts/clusters/s-zmadureira1/code/Users/s-zmadureira/image_splits_MELandBEN"

train_dir = os.path.join(base_dir, 'Training')
test_dir = os.path.join(base_dir, 'Testing')
validate_dir = os.path.join(base_dir, 'validate')
img_size=224

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=5,
    zoom_range = 0.1 ,
    horizontal_flip=False,
    channel_shift_range=.001)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
                  batch_size=16,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validate_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=16,
    class_mode='binary')

from tensorflow.keras.applications.resnet50 import ResNet50

# create an instance of the ResNet50 model
base_model = ResNet50(weights='imagenet')
for layer in base_model.layers:
    layer.trainable = False

model2 = models.Sequential()
model2.add(base_model)
model2.add(layers.Flatten())
model2.add(layers.Dense(256, activation="relu"))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(512, activation="relu"))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(1024, activation="relu"))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(2048, activation="relu"))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(4096, activation="relu"))
model2.add(layers.Dropout(0.5))
model2.add(layers.Dense(256, activation="relu"))
model2.add(layers.Dropout(0.5))
model2.add(layers.BatchNormalization())
model2.add(layers.Dense(1, activation='sigmoid'))

from tensorflow.keras import metrics
opt = tf.keras.optimizers.Adam(
    learning_rate=2e-5,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07,
    amsgrad=True,
    name="Adam")
from keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint('bestmodeltflearning_resnet_pre_finetune.h5', monitor='val_recall_4', verbose=1,
                             save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model2.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=[metrics.Recall(),metrics.Precision(),'accuracy', metrics.AUC(name = 'AUC'),metrics.AUC(curve='PR', name = 'PR-AUC')
                       ,metrics.FalsePositives(), metrics.FalseNegatives(), metrics.TrueNegatives(),metrics.TruePositives()])


history = model2.fit(
    train_generator,
    epochs=25,
    validation_data=validation_generator, verbose=1,callbacks=callbacks_list)

Found 12166 images belonging to 2 classes.
Found 1539 images belonging to 2 classes.
Found 1521 images belonging to 2 classes.
Epoch 1/25
761/761 [==============================] - ETA: 0s - loss: 0.6947 - recall_4: 0.4625 - precision_4: 0.4821 - accuracy: 0.5063 - AUC: 0.5076 - PR-AUC: 0.4850 - false_positives_4: 2885.0000 - false_negatives_4: 3121.0000 - true_negatives_4: 3474.0000 - true_positives_4: 2686.0000
Epoch 1: val_recall_4 improved from -inf to 0.00000, saving model to bestmodeltflearning_resnet_pre_finetune.h5
761/761 [==============================] - 1819s 2s/step - loss: 0.6947 - recall_4: 0.4625 - precision_4: 0.4821 - accuracy: 0.5063 - AUC: 0.5076 - PR-AUC: 0.4850 - false_positives_4: 288

In [ ]:
# reset our data generators
train_generator.reset()
validation_generator.reset()
# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in base_model.layers[15:]:
	layer.trainable = True

print("[INFO] re-compiling model...")

model.compile(loss="categorical_crossentropy", optimizer=tf.keras.optimizers.RMSprop(lr=1e-4),
	metrics=["accuracy"])
# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
checkpoint = ModelCheckpoint('bestmodeltflearning_resnet_post_finetune.h5', monitor='val_recall_4', verbose=1,
                             save_best_only=True, mode='max')

callbacks_list = [checkpoint]
H = model.fit(
	x=train_generator,
	validation_data=validation_generator,
    callbacks=[checkpoint],

	epochs=20
)